In [ ]:
srun --job-name=LLTM -G1 --pty \
    bash -c "source /data/ai_club/team_3_2024-25/team3-env-finetune/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

2025-03-23 23:56:00.631194: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 23:56:00.644262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742788560.660596 1392256 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742788560.665639 1392256 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 23:56:00.681754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# model_id = "microsoft/Phi-4-mini-instruct" # ms claims it knows finnish, but FAKE!!!
model_id = "utter-project/EuroLLM-9B-Instruct"

bnb = BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_8bit_use_double_quant=True,
    # bnb_8bit_quant_type="nf8",
    # bnb_8bit_compute_dtype=torch.bfloat16,

    # llm_int8_enable_fp32_cpu_offload=True
)

# TOKEN = ''

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=TOKEN
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    low_cpu_mem_usage=True,
    # attn_implementation="flash_attention_2",
    torch_dtype=torch.float16,
    quantization_config=bnb,
    token=TOKEN
)

config.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.4k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.33G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [3]:
tokenizer.decode(
    model.generate(
        **tokenizer('Oletko', return_tensors='pt').to('cuda'),
        max_new_tokens=10
    )[0]
)

Setting `pad_token_id` to `eos_token_id`:4 for open-end generation.


'Oletko koskaan ajatellut, että joku muu voisi nähdä'

In [5]:
vocab = [
    'terve', 'hei', 'talo', 'vesi', 'ystävä', 'huomenta', 'velho', 'suomi', 'koira', 'nimi', 'nimeni', 'nimesi', 'nimensä',
    'ystäväni', 'ystäväsi', 'ystävänsä', 'vanha', 'hyvää', 'suomalainen', 'mukava', 'minä', 'minun', 'olen', 'olenko', 'sinä', 'sinun', 'olet',
    'oletko', 'hän', 'hänen', 'on', 'onko', 'matti', 'aleksi', 'sami', 'kyllä', 'ei', 'mitä', 'mikä', 'kuka', 'rossi', 'lucas'
]

In [21]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
 
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0,
    "do_sample": False
}

pipe(
    [
        # {"role": "system", "content": f"You are a Finnish language teacher.\n\n'IMPORTANT: Your responses must only use words in this allowed vocab: {vocab} and any emoji/punctuation."},
        # {"role": "user", "content": "Use the words in the allowed vocab to ask if I have something"}

        # {"role": "system", "content": f"You are a Finnish language teacher."},
        # {"role": "user", "content": "What are the mistakes in this sentence:\nSyön jäätelö"}
      
        # {"role": "system", "content": f"You are a Finnish language teacher."},
        # {"role": "user", "content": "What are the mistakes in this sentence:\nTämä on minun koirasi"}

        {"role": "system", "content": f"Sinä olet suomen kielen opettaja."},
        {"role": "user", "content": "Mitkä ovat tämän lauseen virheet, jos niitä on?\nTämä on minun koirasi"}
    ],
    **generation_args
)

Device set to use cuda:0
/data/ai_club/team_3_2024-25/team3-env-finetune/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[{'generated_text': 'Tässä lauseessa ei ole virheitä. Se on oikein muodostettu ja selkeä. Käännös on myös oikein: "This is my dog."\n\nJos haluat tehdä lauseesta hieman monipuolisemman, voit lisätä siihen lisää yksityiskohtia, kuten koiran nimen tai ikää. Esimerkiksi: "Tämä on minun koirani, jonka nimi on Max ja joka on 3-vuotias."'}]

---

In [14]:
import tiktoken

from openai import OpenAI

client = OpenAI(
    base_url = "http://dh-dgxh100-2.hpc.msoe.edu:8000/v1",
    api_key = "not_used"
)

out = client.chat.completions.create(
    model="meta/llama-3.1-70b-instruct",
    messages=[
        { "role": "system", "content": "You are a Finnish teacher." },
        {
            "role": "user",
            "content": "What is the most obvious mistake in this sentence (if it has one) \"Tämä on minun koirani\"? Respond in JSON {'correct_sentence': str, 'breif explanation': str|'NA'}.",
        },
    ],
    max_tokens=1024,
    stream=True,
    temperature=0,
    response_format={'type': 'json_object'}
)

out_str = ''
for t in out:
    tok = t.choices[0].delta.content
    if not tok: continue
    print(tok, end='')
    out_str += tok



{"correct_sentence": "Tämä on minun koirani", "breif explanation": "NA"}

 

 

 

 

In [19]:
import json
json.loads(out_str+'}')

JSONDecodeError: Extra data: line 9 column 2 (char 84)

In [9]:
out = client.chat.completions.create(
    model="meta/llama-3.1-70b-instruct",
    messages=[
        { "role": "system", "content": "You are a Finnish teacher." },
        {
            "role": "user",
            "content": "What is the most obvious mistake in this sentence (if it has one) \"Tämä on minun koirani\"?",
        },
    ],
    max_tokens=100
)

out


ChatCompletion(id='chat-224c2af44cd046ba8a2f4706dc524330', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='In Finnish, the correct sentence would be "Tämä on minun koirani" actually has one mistake that is quite common for non-native speakers.\n\nThe mistake is the use of "minun" instead of "omani". "Minun" is the genitive form of the pronoun "minä", whereas "omani" is the possessive form.\n\nSo, the corrected sentence would be: "Tämä on omani koira."\n\nHowever, it\'s worth noting', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None), stop_reason=None)], created=1742499135, model='meta/llama-3.1-70b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=100, prompt_tokens=48, total_tokens=148, completion_tokens_details=None, prompt_tokens_details=None))

In [10]:
out.choices[0].message.content

'In Finnish, the correct sentence would be "Tämä on minun koirani" actually has one mistake that is quite common for non-native speakers.\n\nThe mistake is the use of "minun" instead of "omani". "Minun" is the genitive form of the pronoun "minä", whereas "omani" is the possessive form.\n\nSo, the corrected sentence would be: "Tämä on omani koira."\n\nHowever, it\'s worth noting'

In [11]:

enc.encode("You are a Finnish teacher.").__len__() + 4 # 4 extra tokens for llama: start_header, end_header, eos, 2 newlines (part of prompt format

NameError: name 'enc' is not defined

In [3]:
from openai import OpenAI
import tiktoken
import getpass
import json
from dataclasses import dataclass

USER = getpass.getuser()

TOKEN_COUNT_PATH = None

client = OpenAI(
    base_url = "http://dh-dgxh100-2.hpc.msoe.edu:8000/v1",
    api_key = "not_used"
)

enc = tiktoken.get_encoding("cl100k_base")

# TODO REMOVE
TOKEN_COUNT_PATH = '/data/ai_club/team_3_2024-25/tokcounts2/'

def _ntoks(text):
    return enc.encode(text).__len__() + 4 # 4 extra tokens for llama: start_header, end_header, eos, 2 newlines (part of prompt format)

def _inc_tok_count(mode, amt):
    if TOKEN_COUNT_PATH is None:
        raise Exception('Set TOKEN_COUNT_PATH before infernece')
    fname = f'{USER}_{mode}.txt'
    try:
        with open(TOKEN_COUNT_PATH+fname, 'r') as f:
            count = int(f.read().strip())
    except FileNotFoundError:
        count = 0
    except ValueError:
        raise Exception(f'Token Count Corrupted: {fname}')
    
    count += amt

    with open(TOKEN_COUNT_PATH+fname, 'w') as f:
        f.write(str(count)+'\n')

@dataclass
class Msg:
    role: str
    content: str
    response_format: list = None

class LLM:
    def __init__(self, sys_prompt:str=None):
        self._hist = []
        self._awaiting_streamed = False

    def _hist_to_prompt(self):
        prompt = []
        tok_count = 0
        for msg in self._hist:
            content = msg.content
            is_last = msg == self._hist[-1]
            if msg.response_format and is_last:
                json_format = {k:'...' for k in msg.response_format}
                content += f'\n\nRespond in this json: {json_format}'
            elif msg.response_format:
                content += '\n\nRespond in JSON.'
            
            tok_count += _ntoks(content)

            prompt.append({
                'role': msg.role,
                'content': content
            })

        return prompt, tok_count

    def _call_default(self, messages, temperature, max_tokens):
        out = client.chat.completions.create(
            model="meta/llama-3.1-70b-instruct",
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
        out = out.choices[0].message.content
        out_toks = _ntoks(out)

        _inc_tok_count('out', out_toks)

        self._hist.append(Msg('assistant', out))

        return out
        
    def _call_stream(self, messages, temperature, max_tokens):
        out = client.chat.completions.create(
            model="meta/llama-3.1-70b-instruct",
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
            stream=True
        )

        self._hist.append(Msg('assistant', ''))
        self._awaiting_streamed = True

        def tok_stream():
            for t in out:
                tok = t.choices[0].delta.content

                if not tok: continue
                _inc_tok_count('out', 1)
                self._hist[-1].content += tok
                yield tok

            _inc_tok_count('out', 4) # 4 exta used in llama prompt format
            self._awaiting_streamed = False

        return tok_stream()

    def _call_fmted(self, messages, temperature, max_tokens, response_format):
        out = client.chat.completions.create(
            model="meta/llama-3.1-70b-instruct",
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
            response_format={'type': 'json_object'}   
        )
        out = out.choices[0].message.content
        out_toks = _ntoks(out)
        _inc_tok_count('out', out_toks)
        
        try:
            out = json.loads(out)
        except:
            raise Exception(f'Bad JSON output. {out} != {resposne_format}')

        if not all(k in out.keys() for k in response_format):
            raise Exception(f'Missing json keys. {out.keys()} != {response_format}')

        return out

    def __call__(self, prompt, response_format:str|list|None=None, temperature=0, max_tokens=1024):
        if self._awaiting_streamed:
            raise Exception('Cannot start a new message before ending a streamed one.')

        is_resp_fmted = type(response_format) is list

        self._hist.append(Msg('user', prompt))
        if is_resp_fmted:
            self._hist[-1].response_format = response_format

        messages, in_toks = self._hist_to_prompt()
        _inc_tok_count('in', in_toks)

        if response_format is None:
            return self._call_default(messages, temperature, max_tokens)
        elif response_format == 'stream':
            return self._call_stream(messages, temperature, max_tokens)
        elif is_resp_fmted:
            return self._call_fmted(messages, temperature, max_tokens, response_format)
        else:
            raise Exception(f'Unsupported Response Format: {response_format}')


In [4]:
l = LLM()

In [9]:
out = l('who you', response_format='stream')

Exception: Cannot start a new message before ending a streamed one.

In [6]:
out

{'name': 'Assistant', 'job': 'AI Conversational Model'}

In [ ]:
out

In [ ]:
l._hist

[Msg(role='user', content='who you', response_format=None),
 Msg(role='assistant', content='', response_format=None)]

In [ ]:
enc.encode('Hello! It seems like we\'re having a friendly echo. How can I assist you today?').__len__() + 4